<a href="https://colab.research.google.com/github/SFIComplexityExplorer/Mesa-ABM-Tutorial/blob/main/Session_8_Sugar_and_Spice_Step_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 引入依赖包

In [1]:

try: 
  import mesa
except: 
  !pip install mesa --quiet
import mesa
import numpy as np
import math
import matplotlib.pyplot as plt

%matplotlib inline

# 资源类 

In [2]:
class Sugar(mesa.Agent):
    '''
    Sugar:
    - 包含一定数量的糖
    - 每轮增长一单位数量的糖
    '''

    def __init__(self, unique_id, model, pos, max_sugar): 
        super().__init__(unique_id, model)
        self.pos = pos
        self.amount = max_sugar
        self.max_sugar = max_sugar


    def step(self):
        '''
        糖每轮增长一单位数量，直到最大值
        '''
        self.amount = min([self.max_sugar, self.amount+1])

In [3]:
class Spice(mesa.Agent):
  '''
  spice:
  - 包含一定数量的香料
  - 每轮增长一单位数量的香料
  '''

  def __init__(self, unique_id, model, pos, max_spice):
    super().__init__(unique_id, model)
    self.pos = pos
    self.amount = max_spice
    self.max_spice = max_spice
  
  def step(self): 
    '''
    香料每轮增长一单位数量，直到最大值
    '''
    self.amount = min([self.max_spice, self.amount+1])

# 交易主体类

In [4]:
class Trader(mesa.Agent): 
    '''
    Trader:
    - 能够代谢糖和香料
    - 收集和交易糖和香料来生存与繁衍
    '''


    def __init__(self, unique_id, model, pos, moore=False, sugar=0, 
                spice=0, metabolism_sugar=0, metabolism_spice=0, 
                vision=0):
        super().__init__(unique_id, model)
        self.pos = pos
        self.moore = moore
        self.sugar = sugar
        self.spice = spice
        self.metabolism_sugar = metabolism_sugar
        self.metabolism_spice = metabolism_spice
        self.vision = vision
    


# Model Class

In [5]:
class SugarscapeG1mt(mesa.Model):
  '''
  通过Traders (GImt)来运行Sugarscape的模型
  来自 Axtell和Epstein《Growing Artifical Societies》
  '''
  
  
  def __init__(self, width=50,height=50, initial_population=200,
               endowment_min=25, endowment_max=50, metabolism_min=1,
               metabolism_max=5, vision_min=1, vision_max=5):
    
    # 初始化糖景的宽度和高度
    self.width = width
    self.height = height 
    # 初始化交易主体属性
    self.initial_population = initial_population
    self.endowment_min = endowment_min
    self.endowment_max = endowment_max
    self.metabolism_min = metabolism_min
    self.metabolism_max = metabolism_max
    self.vision_min = vision_min
    self.vision_max = vision_max

    # 初始化mesa的日程表
    self.schedule = mesa.time.RandomActivationByType(self)    
    # 初始化mesa的网格类
    self.grid = mesa.space.MultiGrid(self.width, self.height, torus=False)

    # 从辅助资料中读取糖和香料的分布
    sugar_distribution =np.genfromtxt("sugar-map.txt")
    spice_distribution = np.flip(sugar_distribution, 1)
    
    agent_id = 0
    for _,x,y in self.grid.coord_iter():
      max_sugar = sugar_distribution[x,y]
      if max_sugar > 0: 
        sugar = Sugar(agent_id, self, (x,y), max_sugar)
        self.schedule.add(sugar)
        self.grid.place_agent(sugar, (x,y))
        agent_id += 1
    
      max_spice = spice_distribution[x,y]
      if max_spice > 0: 
        spice = Spice(agent_id, self, (x,y), max_spice)
        self.schedule.add(spice)
        self.grid.place_agent(spice, (x,y))
        agent_id += 1    

    for i in range(self.initial_population):
      # 得到随机的坐标
      x = self.random.randrange(self.width)
      y = self.random.randrange(self.height)
      # see Growing Artificial Societies p. 108 for initialization
      # 给予主体初始的禀赋
      sugar = int(self.random.uniform(self.endowment_min, self.endowment_max+1))
      spice = int(self.random.uniform(self.endowment_min, self.endowment_max+1))
      # 给予主体初始代谢
      metabolism_sugar = int(self.random.uniform(self.metabolism_min, self.metabolism_max+1))
      metabolism_spice = int(self.random.uniform(self.metabolism_min, self.metabolism_max+1))
      # 给予主体视野
      vision = int(self.random.uniform(self.vision_min, self.vision_max+1))
      # 实例化交易主体
      trader = Trader(agent_id, 
                      self,
                      (x,y),
                      moore = False, 
                      sugar = sugar, 
                      spice = spice, 
                      metabolism_sugar = metabolism_sugar, 
                      metabolism_spice = metabolism_spice, 
                      vision = vision)
      # 放置主体到网格中
      self.grid.place_agent(trader, (x,y))
      self.schedule.add(trader)
      agent_id += 1

  def step(self):
    '''
    独特的step函数，可分阶段激活糖和香料 然后随机激活交易主体
    '''    

    # step Sugar agents
    for sugar in self.schedule.agents_by_type[Sugar].values(): 
      sugar.step()
    
    # step Spice agents
    for spice in self.schedule.agents_by_type[Spice].values(): 
      spice.step()

    self.schedule.steps += 1 # 对于数据收集器(data collector)跟踪步数很重要

  def run_model(self, step_count=1000):

    for i in range(step_count):
      print(i)
      self.step()

# Run Sugarscape

In [6]:
model = SugarscapeG1mt()
model.run_model(step_count=5)


0
1
2
3
4
